# AgriFrost-AI 教程: 端到端霜冻预测

<div align="center">

<img src="../docs/logo/AgriFrost-AI-transparent.png" alt="AgriFrost-AI Logo" width="150"/>

## 🌡️ AgriFrost-AI 完整工作流演示

**从数据加载到模型训练再到预测生成的完整示例**

*F3 Innovate Frost Risk Forecasting Challenge (2025)*

</div>

---

## 📋 本教程内容

本 Notebook 将带您完成以下步骤：

1. **环境设置和数据加载**
2. **数据探索和可视化**
3. **特征工程演示**
4. **模型训练（LightGBM）**
5. **模型评估和可视化**
6. **生成预测**

**预计时间**: ~30-60 分钟（取决于数据大小和硬件）

**要求**:
- Python 3.10+
- 已安装项目依赖（`pip install -r requirements.txt`）
- 数据已下载到 `data/raw/frost-risk-forecast-challenge/`



In [ ]:
# 1. 导入必要的库
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# 添加项目根目录到路径
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import json

# 设置显示选项
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

# 设置绘图风格（兼容不同版本的 matplotlib）
try:
    plt.style.use('seaborn-v0_8-darkgrid')
except OSError:
    try:
        plt.style.use('seaborn-darkgrid')
    except OSError:
        plt.style.use('default')
sns.set_palette("husl")

print("✅ 库导入成功！")
print(f"📁 项目根目录: {project_root}")
print(f"🐍 Python 版本: {sys.version.split()[0]}")



## 1. 数据加载和探索

首先，让我们加载原始数据并探索其结构。


In [ ]:
# 1.1 加载原始数据
from src.data.loaders import DataLoader

data_path = project_root / "data/raw/frost-risk-forecast-challenge/cimis_all_stations.csv.gz"

if not data_path.exists():
    print(f"❌ 数据文件未找到: {data_path}")
    print("请先下载数据（参考 docs/QUICK_START.md）")
else:
    print(f"📂 加载数据: {data_path}")
    loader = DataLoader()
    df_raw = loader.load_raw_data(data_path)
    print(f"✅ 数据加载成功！")
    print(f"   形状: {df_raw.shape}")
    print(f"   列数: {len(df_raw.columns)}")
    print(f"   时间范围: {df_raw['Date'].min()} 到 {df_raw['Date'].max()}")
    print(f"   站点数: {df_raw['Stn Id'].nunique()}")


In [ ]:
# 1.2 查看数据概览
if 'df_raw' in locals():
    print("📊 数据概览:")
    print(df_raw.head(10))
    print("\n📋 数据信息:")
    print(df_raw.info())
    print("\n📈 描述性统计:")
    print(df_raw.describe())


## 2. 数据可视化

让我们可视化一些关键模式和特征。


In [ ]:
# 2.1 时间序列可视化
if 'df_raw' in locals():
    # 转换日期列为 datetime
    df_raw['Date'] = pd.to_datetime(df_raw['Date'])
    
    # 选择单个站点进行可视化（例如站点 2）
    df_station = df_raw[df_raw['Stn Id'] == 2].copy()
    df_station = df_station.sort_values('Date')
    
    # 取最近 1000 行用于快速可视化
    df_sample = df_station.tail(1000)
    
    fig, axes = plt.subplots(2, 1, figsize=(14, 10))
    
    # 温度时间序列
    axes[0].plot(df_sample['Date'], df_sample['Air Temp (C)'], label='Air Temperature', linewidth=1)
    axes[0].axhline(y=0, color='r', linestyle='--', label='Frost Threshold (0°C)')
    axes[0].set_xlabel('Date')
    axes[0].set_ylabel('Temperature (°C)')
    axes[0].set_title('Air Temperature Time Series (Station 2, Last 1000 Hours)')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # 相对湿度时间序列
    axes[1].plot(df_sample['Date'], df_sample['Rel Hum (%)'], label='Relative Humidity', color='green', linewidth=1)
    axes[1].set_xlabel('Date')
    axes[1].set_ylabel('Relative Humidity (%)')
    axes[1].set_title('Relative Humidity Time Series (Station 2, Last 1000 Hours)')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"📊 可视化完成！显示站点 2 最近 1000 小时的数据")


In [ ]:
# 2.2 霜冻事件统计
if 'df_raw' in locals():
    # 识别霜冻事件（≤0°C）
    df_raw['is_frost'] = (df_raw['Air Temp (C)'] <= 0.0).astype(int)
    
    # 按月份统计霜冻事件
    df_raw['Month'] = pd.to_datetime(df_raw['Date']).dt.month
    frost_by_month = df_raw.groupby('Month')['is_frost'].agg(['sum', 'count', 'mean'])
    frost_by_month.columns = ['Frost Events', 'Total Observations', 'Frost Rate']
    
    print("📊 每月霜冻事件统计:")
    print(frost_by_month)
    
    # 可视化
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # 霜冻率按月
    axes[0].bar(frost_by_month.index, frost_by_month['Frost Rate'] * 100, color='steelblue')
    axes[0].set_xlabel('Month')
    axes[0].set_ylabel('Frost Rate (%)')
    axes[0].set_title('Frost Rate by Month')
    axes[0].set_xticks(range(1, 13))
    axes[0].grid(True, alpha=0.3, axis='y')
    
    # 霜冻事件数按月
    axes[1].bar(frost_by_month.index, frost_by_month['Frost Events'], color='coral')
    axes[1].set_xlabel('Month')
    axes[1].set_ylabel('Number of Frost Events')
    axes[1].set_title('Total Frost Events by Month')
    axes[1].set_xticks(range(1, 13))
    axes[1].grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n❄️ 总霜冻事件: {df_raw['is_frost'].sum():,}")
    print(f"📊 霜冻率: {df_raw['is_frost'].mean()*100:.2f}%")


## 3. 数据处理管道

现在让我们使用统一的数据处理管道来清洗数据、生成特征和标签。


In [ ]:
# 3.1 配置数据处理管道
from src.data import DataPipeline

# 配置（使用 Top 175 特征）
config = {
    "cleaning": {
        "config_path": str(project_root / "config/data_cleaning.yaml")
    },
    "labels": {
        "threshold": 0.0  # 霜冻阈值：0°C
    },
    "feature_engineering": {
        "enabled": True,
        "feature_selection": {
            "method": "top_k",
            "top_k": 175  # 使用 Top 175 特征
        }
    },
    "random_state": 42
}

print("⚙️ 配置数据管道...")
pipeline = DataPipeline(config=config)
print("✅ 数据管道创建成功！")


In [ ]:
# 3.2 处理数据（使用采样以加快演示速度）
if 'data_path' in locals() and data_path.exists():
    print("🔄 开始数据处理...")
    print("   ⚠️  注意：为了演示速度，我们使用采样数据（10万行）")
    print("   💡 实际训练时可以移除 sample_size 参数使用全部数据")
    
    # 处理数据（使用采样）
    dataset_bundle = pipeline.run(
        data_path=data_path,
        horizons=[12],  # 只处理 12h 时间范围
        use_feature_engineering=True,
        sample_size=100000,  # 采样 10 万行用于演示
        random_state=42
    )
    
    df_processed = dataset_bundle.data
    print(f"✅ 数据处理完成！")
    print(f"   处理后形状: {df_processed.shape}")
    print(f"   特征数: {len(dataset_bundle.feature_columns)}")
    print(f"   标签数: {len(dataset_bundle.label_columns)}")
    
    # 显示特征列
    print(f"\n📋 特征列示例（前 20 个）:")
    for i, feat in enumerate(dataset_bundle.feature_columns[:20]):
        print(f"   {i+1}. {feat}")
    if len(dataset_bundle.feature_columns) > 20:
        print(f"   ... (共 {len(dataset_bundle.feature_columns)} 个特征)")
else:
    print("❌ 数据文件未找到，跳过数据处理步骤")


## 4. 模型训练

现在让我们训练一个 LightGBM 模型。


In [ ]:
# 4.1 准备训练数据
if 'df_processed' in locals():
    from src.training.data_preparation import prepare_features_and_targets
    from src.evaluation.validators import CrossValidator
    from src.models.registry import get_model_class
    
    # 步骤 1: 时间序列分割（避免数据泄露）
    # 注意：必须先分割数据，再准备特征和标签
    print("📊 执行时间序列分割...")
    train_df, val_df, test_df = CrossValidator.time_split(
        df=df_processed,
        train_ratio=0.7,
        val_ratio=0.15,
        date_col="Date"
    )
    
    print(f"   训练集: {len(train_df)} 样本")
    print(f"   验证集: {len(val_df)} 样本")
    print(f"   测试集: {len(test_df)} 样本")
    
    # 步骤 2: 为每个分割准备特征和标签（12h 时间范围）
    print("\n🔧 准备训练集特征和标签...")
    X_train, y_frost_train, y_temp_train = prepare_features_and_targets(
        df=train_df,
        horizon=12,
        track="top175_features"
    )
    
    print("🔧 准备验证集特征和标签...")
    X_val, y_frost_val, y_temp_val = prepare_features_and_targets(
        df=val_df,
        horizon=12,
        track="top175_features"
    )
    
    print("🔧 准备测试集特征和标签...")
    X_test, y_frost_test, y_temp_test = prepare_features_and_targets(
        df=test_df,
        horizon=12,
        track="top175_features"
    )
    
    print("\n✅ 数据准备完成！")
    print(f"   训练集: {X_train.shape[0]} 样本, {X_train.shape[1]} 特征")
    print(f"   验证集: {X_val.shape[0]} 样本")
    print(f"   测试集: {X_test.shape[0]} 样本")
    print(f"   霜冻事件 (训练集): {y_frost_train.sum()} ({y_frost_train.mean()*100:.2f}%)")
    print(f"   平均温度 (训练集): {y_temp_train.mean():.2f}°C")
else:
    print("❌ 跳过模型训练（数据未处理）")


In [ ]:
# 4.2 训练分类模型（霜冻概率预测）
if 'X_train' in locals():
    print("🤖 训练霜冻分类模型 (LightGBM)...")
    
    # 获取模型类
    ModelClass = get_model_class('lightgbm')
    
    # 创建模型实例
    frost_model = ModelClass(
        task_type='classification',
        model_params={
            'n_estimators': 100,  # 演示用较少树数，实际可以更多
            'learning_rate': 0.05,
            'max_depth': 7,
            'random_state': 42,
            'verbosity': -1
        }
    )
    
    # 训练模型
    frost_model.fit(
        X=X_train,
        y=y_frost_train,
        validation_data=(X_val, y_frost_val)
    )
    
    print("✅ 分类模型训练完成！")
    
    # 训练回归模型（温度预测）
    print("🤖 训练温度回归模型 (LightGBM)...")
    
    temp_model = ModelClass(
        task_type='regression',
        model_params={
            'n_estimators': 100,
            'learning_rate': 0.05,
            'max_depth': 7,
            'random_state': 42,
            'verbosity': -1
        }
    )
    
    temp_model.fit(
        X=X_train,
        y=y_temp_train,
        validation_data=(X_val, y_temp_val)
    )
    
    print("✅ 回归模型训练完成！")
else:
    print("❌ 跳过模型训练（数据未准备）")


## 5. 模型评估和可视化

让我们评估模型的性能并可视化结果。


In [ ]:
# 5.1 评估分类模型
if 'frost_model' in locals():
    from src.evaluation.metrics import MetricsCalculator
    
    # 生成预测
    y_frost_pred = frost_model.predict(X_test)
    y_frost_proba = frost_model.predict_proba(X_test)
    
    # 计算指标
    metrics_calc = MetricsCalculator()
    class_metrics = metrics_calc.calculate_classification_metrics(
        y_true=y_frost_test,
        y_pred=y_frost_pred,
        y_proba=y_frost_proba
    )
    
    print("📊 分类模型性能 (测试集):")
    print(f"   ROC-AUC: {class_metrics['roc_auc']:.4f}")
    print(f"   PR-AUC: {class_metrics['pr_auc']:.4f}")
    print(f"   Brier Score: {class_metrics['brier_score']:.4f}")
    print(f"   ECE: {class_metrics['ece']:.4f}")
    print(f"   Accuracy: {class_metrics['accuracy']:.4f}")
    print(f"   Precision: {class_metrics['precision']:.4f}")
    print(f"   Recall: {class_metrics['recall']:.4f}")
    print(f"   F1 Score: {class_metrics['f1_score']:.4f}")
else:
    print("❌ 跳过分类评估（模型未训练）")


In [ ]:
# 5.2 评估回归模型
if 'temp_model' in locals():
    # 生成预测
    y_temp_pred = temp_model.predict(X_test)
    
    # 计算指标
    reg_metrics = metrics_calc.calculate_regression_metrics(
        y_true=y_temp_test,
        y_pred=y_temp_pred
    )
    
    print("📊 回归模型性能 (测试集):")
    print(f"   MAE: {reg_metrics['mae']:.4f}°C")
    print(f"   RMSE: {reg_metrics['rmse']:.4f}°C")
    print(f"   R²: {reg_metrics['r2']:.4f}")
    print(f"   MAPE: {reg_metrics.get('mape', 'N/A')}")
else:
    print("❌ 跳过回归评估（模型未训练）")


In [ ]:
# 5.3 可视化预测结果
if 'y_temp_pred' in locals() and 'y_frost_proba' in locals():
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. 温度预测 vs 真实值
    axes[0, 0].scatter(y_temp_test, y_temp_pred, alpha=0.5, s=10)
    axes[0, 0].plot([y_temp_test.min(), y_temp_test.max()], 
                    [y_temp_test.min(), y_temp_test.max()], 
                    'r--', lw=2, label='Perfect Prediction')
    axes[0, 0].set_xlabel('True Temperature (°C)')
    axes[0, 0].set_ylabel('Predicted Temperature (°C)')
    axes[0, 0].set_title(f'Temperature Prediction (R² = {reg_metrics["r2"]:.4f})')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. 温度预测误差分布
    temp_errors = y_temp_pred - y_temp_test
    axes[0, 1].hist(temp_errors, bins=50, edgecolor='black', alpha=0.7)
    axes[0, 1].axvline(x=0, color='r', linestyle='--', linewidth=2)
    axes[0, 1].set_xlabel('Prediction Error (°C)')
    axes[0, 1].set_ylabel('Frequency')
    axes[0, 1].set_title(f'Temperature Prediction Error Distribution (MAE = {reg_metrics["mae"]:.4f}°C)')
    axes[0, 1].grid(True, alpha=0.3, axis='y')
    
    # 3. ROC 曲线
    from sklearn.metrics import roc_curve, auc
    fpr, tpr, _ = roc_curve(y_frost_test, y_frost_proba)
    roc_auc = auc(fpr, tpr)
    axes[1, 0].plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
    axes[1, 0].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random')
    axes[1, 0].set_xlabel('False Positive Rate')
    axes[1, 0].set_ylabel('True Positive Rate')
    axes[1, 0].set_title('ROC Curve for Frost Classification')
    axes[1, 0].legend(loc="lower right")
    axes[1, 0].grid(True, alpha=0.3)
    
    # 4. 霜冻概率分布
    axes[1, 1].hist(y_frost_proba[y_frost_test == 0], bins=50, alpha=0.7, label='No Frost', color='blue')
    axes[1, 1].hist(y_frost_proba[y_frost_test == 1], bins=50, alpha=0.7, label='Frost', color='red')
    axes[1, 1].set_xlabel('Predicted Frost Probability')
    axes[1, 1].set_ylabel('Frequency')
    axes[1, 1].set_title('Frost Probability Distribution')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()
    
    print("✅ 可视化完成！")
else:
    print("❌ 跳过可视化（预测结果未生成）")


## 6. 特征重要性分析

让我们查看哪些特征对模型最重要。


In [ ]:
# 6.1 获取特征重要性
if 'frost_model' in locals() and 'X_train' in locals():
    try:
        # 获取特征重要性（使用 LightGBM booster API）
        # 注意: LightGBM 需要使用 booster_.feature_importance() 方法
        feature_importance = frost_model.model.booster_.feature_importance(importance_type='gain')
        importance_df = pd.DataFrame({
            'feature': X_train.columns.tolist(),  # 使用训练集的特征列
            'importance': feature_importance
        }).sort_values('importance', ascending=False)
        
        # 显示 Top 20 重要特征
        print("🔝 Top 20 最重要特征:")
        print(importance_df.head(20).to_string(index=False))
        
        # 可视化 Top 20 特征重要性
        fig, ax = plt.subplots(figsize=(12, 8))
        top_features = importance_df.head(20)
        ax.barh(range(len(top_features)), top_features['importance'].values, color='steelblue')
        ax.set_yticks(range(len(top_features)))
        ax.set_yticklabels(top_features['feature'].values)
        ax.set_xlabel('Feature Importance (Gain)')
        ax.set_title('Top 20 Feature Importance (Frost Classification Model)')
        ax.invert_yaxis()  # 最重要的在顶部
        plt.tight_layout()
        plt.show()
        
    except Exception as e:
        print(f"⚠️  无法获取特征重要性: {e}")
        print("   这可能是因为模型类型不支持或模型未正确初始化")
        print("   尝试使用 feature_importances_ 属性:")
        try:
            # 备用方法：使用 feature_importances_ 属性
            feature_importance = frost_model.model.feature_importances_
            importance_df = pd.DataFrame({
                'feature': X_train.columns.tolist(),
                'importance': feature_importance
            }).sort_values('importance', ascending=False)
            print("\n✅ 使用备用方法成功获取特征重要性:")
            print(importance_df.head(20).to_string(index=False))
        except Exception as e2:
            print(f"   备用方法也失败: {e2}")
else:
    print("❌ 跳过特征重要性分析（模型未训练）")


## 7. 生成预测

最后，让我们使用训练好的模型生成新的预测。


In [ ]:
# 7.1 准备新数据进行预测
if 'X_test' in locals() and 'frost_model' in locals():
    # 使用测试集的一部分作为新数据
    new_data = X_test[:100].copy()  # 取前 100 个样本
    
    # 生成预测
    frost_proba_predictions = frost_model.predict_proba(new_data)
    temp_predictions = temp_model.predict(new_data)
    
    # 创建预测结果 DataFrame
    predictions_df = pd.DataFrame({
        'Frost_Probability': frost_proba_predictions,
        'Temperature_Prediction_C': temp_predictions,
        'Frost_Risk': ['Low' if p < 0.1 else 'Medium' if p < 0.5 else 'High' for p in frost_proba_predictions]
    })
    
    print("📊 预测结果示例 (前 20 个):")
    print(predictions_df.head(20).to_string(index=True))
    
    # 统计高风险预测
    high_risk = (predictions_df['Frost_Probability'] > 0.5).sum()
    print(f"\n⚠️  高风险预测 (概率 > 0.5): {high_risk} / {len(predictions_df)} ({high_risk/len(predictions_df)*100:.1f}%)")
    
else:
    print("❌ 跳过预测生成（模型或数据不可用）")


In [ ]:
# 7.2 可视化预测结果
if 'predictions_df' in locals():
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # 霜冻概率分布
    axes[0].hist(predictions_df['Frost_Probability'], bins=30, edgecolor='black', alpha=0.7, color='steelblue')
    axes[0].axvline(x=0.5, color='r', linestyle='--', linewidth=2, label='High Risk Threshold (0.5)')
    axes[0].set_xlabel('Frost Probability')
    axes[0].set_ylabel('Frequency')
    axes[0].set_title('Distribution of Frost Probability Predictions')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3, axis='y')
    
    # 温度预测分布
    axes[1].hist(predictions_df['Temperature_Prediction_C'], bins=30, edgecolor='black', alpha=0.7, color='coral')
    axes[1].axvline(x=0, color='r', linestyle='--', linewidth=2, label='Frost Threshold (0°C)')
    axes[1].set_xlabel('Predicted Temperature (°C)')
    axes[1].set_ylabel('Frequency')
    axes[1].set_title('Distribution of Temperature Predictions')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()
    
    print("✅ 预测可视化完成！")
else:
    print("❌ 跳过可视化（预测结果不可用）")


## 8. 总结和下一步

恭喜！您已经完成了 AgriFrost-AI 的完整工作流程演示！

### 📋 本教程总结

✅ **已完成**:
1. 数据加载和探索
2. 数据可视化和统计分析
3. 数据清洗和特征工程
4. 模型训练（分类和回归）
5. 模型评估和性能分析
6. 特征重要性分析
7. 预测生成和可视化

### 🚀 下一步建议

1. **尝试不同模型**:
   - XGBoost: `get_model_class('xgboost')`
   - CatBoost: `get_model_class('catboost')`
   - LSTM: `get_model_class('lstm')` (需要 GPU)

2. **尝试不同时间范围**:
   - 修改 `horizons=[3, 6, 12, 24]` 训练多个时间范围

3. **尝试不同特征集**:
   - 完整特征集（298 特征）
   - 自定义特征选择

4. **空间聚合**:
   - 尝试 Matrix Cell C/D（多站点特征）
   - 尝试 Matrix Cell E（图神经网络）

5. **模型调优**:
   - 使用超参数优化
   - 尝试不同的模型配置

### 📚 深入学习

- 📖 **快速开始**: `docs/QUICK_START.md`
- 🏗️ **实现指南**: `docs/IMPLEMENTATION_GUIDE.md` / `docs/IMPLEMENTATION_GUIDE_CN.md`
- 🔬 **技术文档**: `docs/TECHNICAL_DOCUMENTATION.md`
- 🤖 **模型指南**: `docs/MODELS_GUIDE.md`
- 📊 **特征指南**: `docs/FEATURE_GUIDE.md`

### 💡 提示

- 使用完整数据集可以获得更好的性能（移除 `sample_size` 参数）
- 增加 `n_estimators` 可以提高模型准确度（但训练时间更长）
- 使用 GPU 可以加速深度学习模型训练
- LOSO 评估可以测试模型的空间泛化能力

---

**感谢使用 AgriFrost-AI！** 🌡️🤖

**文档版本**: 1.0  
**最后更新**: 2025-11-19  
**作者**: Zhengkun LI (TRIC Robotics / UF ABE)

